# Advanced Lane Finding 


Stages:

1. Calculate Camera Calibration Matrix

2. Apply Distortion Correction
3. Perform appropriate colour transformations 
4. Adjust perspective to get Bird's Eye View
5. Perform Lane Detection
6. Calculate curvature
7. Warp back to original image
8. Output Image

1 is a pre-processing step, while 2-8 indicate our processing pipeline

In [1]:
## Import Libraries

import pickle
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


In [2]:
# Calculate Camera Distortion Matrix

# This is performed in a seperate notebook and the numbers brought in.
dist_pickle = pickle.load( open( "cameracal.p", "rb" ) )

mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]

# undist = cv2.undistort(img, mtx, dist, None, mtx)

In [3]:
# PipeLine

def Pipeline(img):
    
    # 2 Distortion
    img = cv2.undistort(img, mtx, dist, None, mtx)
    
    # 3 Colour Transforms
    
    
    # 4 Adjust to Bird's Eye
    
    
    # 5 Lane Detectin
    
    
    # 6 Calculate Curvature
    

    # 7 Warp back to Original
    
    
    # 8 return image
    return img
    

[[  1.03916946e+03   0.00000000e+00   6.62324897e+02]
 [  0.00000000e+00   1.05184935e+03   2.40415863e+02]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00]]
[[ -4.28707372e-01   1.52863667e+00   1.63983842e-02  -6.13484269e-04
   -3.08045017e+00]]
